In [5]:
#importing relevant libraries
import pandas as pd
import statsmodels.api as sm
import scipy
import matplotlib.pyplot as pyplot
from scipy import stats
import numpy as np
from pandas.core.nanops import nanmean as pd_nanmean
from statsmodels.tsa.seasonal import seasonal_decompose

df=pd.read_csv("C:\Users\Rawal Rawal\Downloads\salesdata.csv") #importing dataset
df.index=pd.to_datetime(df['Date']) #setting a datetime index,to obtain a timeseries object
series = df['Sales'] 
df.fillna(0)  #assigning all missing values as zero

,Date,Sales,Unnamed: 2,Unnamed: 3,Unnamed: 4
Date,,,,,
2007-01-01,1/1/2007,4600,0.0,0.0,=
2007-01-02,1/2/2007,5700,0.0,0.0,0
2007-01-03,1/3/2007,6800,0.0,0.0,0
2007-01-04,1/4/2007,7900,0.0,0.0,0
2007-01-05,1/5/2007,9000,0.0,0.0,0
2007-01-06,1/6/2007,10100,0.0,0.0,0
2007-01-07,1/7/2007,5700,0.0,0.0,0
2007-01-08,1/8/2007,6800,0.0,0.0,0
2007-01-09,1/9/2007,7900,0.0,0.0,0


In [6]:
#using seasonal decomposiiton to obtain trend,seasonality and residuals

result = seasonal_decompose(series, model='additive',extrapolate_trend=6) 
result.resid = np.array(result.resid)
result.resid = map(lambda x:x+3000, result.resid) #transforming values so that box-cox can be used
result.seasonal = np.array(result.seasonal)
result.trend = np.array(result.trend)

trendeffects = result.trend[-24:]    #trend effect to be added to forecast
seasonalityeffects = result.seasonal[-24:]   #seasonality effect to be added to forecast


In [7]:
test =  df[-24:] #datset used for checking forecast accuracy
train = df[0:-24]   #dataset used for modelling

power = scipy.stats.boxcox_normmax(result.resid, method='mle')
power = round(power,2)#the power that will be used to make data stationary
 
result.resid =  np.array(result.resid)
power = float(power)
result4 = (result.resid**power - 1)/power              #box-cox transform
result4 = np.array(result4)
result4 = result4[:-24]
result4 = pd.DataFrame(result4, index=train.index,columns = ['val'])  #converting results to dataframe
forecast = []        #forecast values will be appended to this
Values = []         #model parameters are appended to this
MAPE = []           #The Mean Absolute Percentage Error is appended to this
Shapirovals = []    #Shapiro Wilk Test Statistic values will be appended to this
result4.head()

,val
Date,
2007-01-01,130.742349
2007-01-02,190.567824
2007-01-03,252.909642
2007-01-04,322.694915
2007-01-05,369.230933


In [16]:
#applying the SARIMAX model
for p in [1,2,3,4,5]:
      for q in [0,1,2,3,4,5]:
            try:                                                       # to omit models that are non-invertible
                mod = sm.tsa.statespace.SARIMAX(result4['val'],order=(p,0,q))
                res = mod.fit(disp=False)
                forecasted = [res.forecast(steps=24).values]  
                testvals = np.array([test['Sales']])       #values in the test series
                
                forecasted = np.array(forecasted)
                forecasted = ((forecasted*0.95)+1)**(1/0.95)       #converting the values back to their pre box-cox form 
                forecasted = forecasted+trendeffects+seasonalityeffects
                mapev = ((testvals-forecasted)*100)/testvals        #Calculating the Absolute Percentage Error
                mapev = np.array(mapev)
                mapev = np.absolute(mapev)
                mape = np.mean(mapev) 
                error = testvals-forecasted                    #Calculating Errors
                W,r = scipy.stats.shapiro(error)                  #Applying the Wilk Shapiro test on the error series
                MAPE.append(mape)
                Shapirovals.append(W)
                Values.append([p,q])
                forecast.append(forecasted)                     #obtaining final lists
            except:
                continue
                
indexapp = np.argmax(Shapirovals)                                #finding the index for which W-S test value is the highest
forecasted = forecast[indexapp]                                  # forecast values corresponding to the index above
print  (MAPE[indexapp])    
print (Shapirovals[indexapp])#MAPE for the chosen model

print  (Values[indexapp])                                        #parameters of the chosen model



C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\Anaconda2\lib\site-packages\statsmodels\tsa\base\tsa_model.py:171: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)
C:\ProgramData\

8.642680188989216
0.93564760685
[3, 5]
